In [11]:
import json
import gzip
import re
import requests

template={}
with gzip.open("jawiki-country.json.gz","r") as data_file:
    for line in data_file:
        data=json.loads(line)
        if data["title"]=="イギリス":
            text=data["text"]
            obj1=re.search(".*基礎情報 国",text)
            obj2=re.search(r"(.*)\n}}\n",text[obj1.end():])
            text2=text[obj1.end():obj2.end()+obj1.end()].split("\n")
            for line2 in text2:
                item=re.match(r"\|(.+)\s=(.*)",line2)
                if item!=None:
                    item2=re.sub(r"\'{2,5}|<br />|<ref(.*)>|\(\+.+?\)","",item.group(2))
                    item2=re.sub(r"\{\{lang\|(?:[^\|]+?\|)([^\|]+?)\}\}",r"\1",item2)
                    item2=re.sub(r"\[\[ファイル:(.+?)\]\]",r"\1",item2)
                    item2=re.sub(r"\[\[([^\|]+?)\]\]",r"\1",item2)
                    item2=re.sub(r"\[\[(?:[^\|]+?\|)([^\|]+?)\]\]",r"\1",item2)
                    item2=re.sub(r"\{\{[^仮リンク]+?\}\}","",item2)
                    item2=re.sub(r"\{\{仮リンク\|([^\|]+?)\|.+?\}\}",r"\1",item2)
                    template[item.group(1)]=item2
                    
                    if item.group(1)=="国旗画像":
                        S=requests.Session()
                        URL="https://en.wikipedia.org/w/api.php"
                        PARAMS = {
                            "action": "query",
                            "format": "json",
                            "prop": "imageinfo",
                            "titles":"File:"+template["国旗画像"],
                            "iiprop":"url"
                        }
                        R=S.get(url=URL,params=PARAMS)
                        DATA = R.json()
                        PAGES = DATA["query"]["pages"]

                        for k, v in PAGES.items():
                            print(v["title"] + " is uploaded by User:" + v["imageinfo"][0]["url"] )

File:Flag of the United Kingdom.svg is uploaded by User:https://upload.wikimedia.org/wikipedia/en/a/ae/Flag_of_the_United_Kingdom.svg
